In [1]:
###############################################################################
# Common parameters
###############################################################################
year_plan, month_plan  = 2024, 4
l_holiday = [29]
l_date_ect_cancel = []

###############################################################################
# Fixed parameters
###############################################################################
# General
year_start, month_start = 2024, 4
# Form
dict_duty = {'ect': 0, 'am': 1, 'pm': 2, 'day': 3, 'ocday': 4, 'night': 5, 'emnight':6, 'ocnight': 7}
dict_duty_jpn = {'am': '午前日直', 'pm': '午後日直', 'day': '日直', 'ocday': '日直OC', 'night': '当直', 'emnight': '救急当直', 'ocnight': '当直OC'}
dict_score_duty = {'duty':         ['am', 'pm', 'day', 'night', 'emnight', 'ocday', 'ocnight', 'ect'],
                   'ampm':         [0.5, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
                   'daynight':     [0.0, 0.0, 1.0, 1.0, 1.5, 0.0, 0.0, 0.0],
                   'ampmdaynight': [0.5, 0.5, 1.0, 1.0, 1.5, 0.0, 0.0, 0.0],
                   'oc':           [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0],
                   'ect':          [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]}
dict_title_duty = {'assoc':            ['ocday', 'ocnight'],
                   'instr':            ['am', 'pm', 'ocday', 'ocnight'],
                   'assist_leader':    ['am', 'pm', 'day', 'night', 'ocday', 'ocnight'],
                   'assist_subleader': ['am', 'pm', 'day', 'night'],
                   'limtermclin':      ['am', 'pm', 'day', 'night'],
                   'stud':             ['day', 'night'],
                   'assist_child':     ['am', 'pm']}
dict_class_duty = {'class': ['ampm', 'ampm', 'daynight_tot', 'daynight_tot', 'daynight_tot', 'night_em', 'night_wd', 'night_wd', 'daynight_hd', 'daynight_hd', 'oc_tot', 'oc_tot', 'oc_day', 'oc_night', 'ect'],
                   'date':  ['all', 'all', 'all', 'all', 'all', 'all', 'wd', 'all', 'all', 'hd', 'all', 'all', 'all', 'all', 'all'],
                   'duty':  ['am', 'pm', 'day', 'night', 'emnight', 'emnight', 'night', 'emnight', 'day', 'night', 'ocday', 'ocnight', 'ocday', 'ocnight', 'ect']}
# Data collection
address_response = "https://docs.google.com/spreadsheets/d/1zCdtuSHwzYn4idJM6ZZzzLJbQh4cJjf8uL8b4FySVCw/edit?resourcekey#gid=1443213949"
# Optimizing assignment count
l_day_ect = [0, 2, 3] # Monday, Wednesday, Thursday
day_em, l_week_em = 2, [] # Wednesday, 1st and 3rd weeks
l_type_score = ['ampm','daynight','ampmdaynight','oc','ect']
l_class_duty = ['ampm','daynight_tot','night_em','night_wd','daynight_hd','oc_tot','oc_day','oc_night','ect']
# Optimizing assignment, parameters for avoiding/penalizing close duties
dict_closeduty = {'daynight': {'l_duty': ['day', 'ocday', 'night', 'emnight', 'ocnight'], 'thr_hard': 1, 'thr_soft': 5},
                  'ect':      {'l_duty': ['ect'],                                         'thr_hard': 1, 'thr_soft': 4},
                  'ampm':     {'l_duty': ['am', 'pm'],                                    'thr_hard': 1, 'thr_soft': 5}}
# Optimizing assignment, parameters for avoiding overlapping duties
ll_avoid_adjacent = [[['pm', 0], ['night', 0], ['emnight', 0], ['ocnight', 0]], [['night', 0], ['emnight', 0], ['ocnight', 0], ['ect', 1], ['am', 1]]]
l_title_fulltime = ['assist'] # ['limterm_instr', 'assist', 'limterm_clin']
# Notification
t_sleep = 600
# Parameters for replacement
sheet_id = "1glzf0fM1jyAZffFE7l7SHE26m3M4QBI5AAOsdSlmHxE"
l_scope = ['https://www.googleapis.com/auth/calendar']

In [3]:
###############################################################################
# Create Google form
###############################################################################
from form import *
d_cal, d_date_duty, s_cnt_duty, s_cnt_class_duty, d_cal_duty, d_form =\
    prepare_form(year_plan, month_plan, l_holiday, l_date_ect_cancel, l_day_ect, day_em, l_week_em, l_class_duty, dict_duty, dict_score_duty, dict_duty_jpn, dict_title_duty, dict_class_duty)

In [4]:
###############################################################################
# Collect google form response
###############################################################################
from collect import *
str_member_missing, str_mail_missing, d_availability, d_info, d_member =\
    collect_availability(year_plan, month_plan, address_response)

Missing members/emails:
藤川 慎也, 越山 太輔, 濱本 優, 金 叡綱, 古川 由己, 田中 李樹, 南 倫, 大熊 彩子, 熊谷 友梨香, 朝重 菜々美
fjkwsny@gmail.com, koshiyamadaisuke@gmail.com, yhamamoto.tky@gmail.com, Primavera9132@gmail.com, furukawa.yuki@gmail.com, antell@hotmail.co.jp, rinmina6@gmail.com, okuma.ayako@gmail.com, eureka.kumagai.1023@gmail.com, mochikura82@gmail.com


In [ ]:
###############################################################################
# Optimize assignment count and assign members
###############################################################################
from assign import *
# Related to assignment count optimization
dict_c_diff_score_current = {'ampm': 0.001, 'daynight': 0.001, 'ampmdaynight': 0.001, 'oc': 0.001, 'ect': 0.01}
#dict_c_diff_score_total = {'ampm': 0.01, 'daynight': 0.01, 'ampmdaynight': 0.01, 'oc': 0.01, 'ect': 0.1}
dict_c_diff_score_total = {'ampm': 0.01, 'daynight': 0.01, 'ampmdaynight': 1.0, 'oc': 0.01, 'ect': 0.1}
# Related to individual assignment
c_assign_suboptimal = 0.0001
#c_cnt_deviation = 0.001
c_cnt_deviation = 0.1
c_closeduty = 0.01
#c_closeduty = 0.1
l_date_duty_fulltime = []
type_limit = 'soft' # 'hard': never exceed, 'soft': outlier penalized, 'ignore': no penalty
l_date_duty_skip_manual = []
#l_date_duty_skip_manual = ['23_'] # All duties starting with 23_
#l_date_duty_skip_manual = ['23_am']

d_assign, d_assign_date_print, d_assign_member, d_deviation, d_score_print, d_closeduty =\
    optimize_count_and_assign(year_plan, month_plan, year_start, month_start,
                              l_type_score, l_class_duty, dict_c_diff_score_current, dict_c_diff_score_total,
                              l_date_duty_skip_manual, dict_closeduty, ll_avoid_adjacent,
                              l_title_fulltime, l_date_duty_fulltime, type_limit,
                              c_assign_suboptimal, c_cnt_deviation, c_closeduty)

In [ ]:
###############################################################################
# Notify google calendar
###############################################################################
from notify import *
notify(year_plan, month_plan, l_scope, t_sleep)

In [ ]:
###############################################################################
# Check availability of certain date_duty
###############################################################################
from check import *
date_duty = '20_night'
check_availability_date_duty(year_plan, month_plan, date_duty)

In [ ]:
###############################################################################
# Check availability of certain member
###############################################################################
from check import *
id_member = 37
check_availability_member(year_plan, month_plan, id_member)

In [ ]:
###############################################################################
# Replace assignment
###############################################################################
from replace import *
# Load from Gforms result and check
d_replace_checked = check_replacement(year_plan, month_plan, sheet_id)
# Apply checked replacement plan
d_assign, d_assign_date_print, d_assign_member, d_deviation, d_deviation_summary, d_score_current, d_score_total, d_score_print =\
    replace_assignment(year_plan, month_plan, l_type_score, l_class_duty, d_replace_checked)
# Add new event to Gcalendar
l_result_event = add_replaced_calendar(year_plan, month_plan, d_replace_checked, l_scope)